# Data Preparation Exercise Reference
------
## 基本作業

請讀取104年花東空品區三個測站的資料，進行分析，並回答以下問題

In [ ]:
%matplotlib inline
# Import libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re

首先，我們從檔案讀取資料，檢查一下資料的維度

In [ ]:
hualian = pd.read_excel('../data/104年花蓮站_20160320.xls')
hualian.shape

In [ ]:
taidong = pd.read_excel('../data/104年臺東站_20160320.xls')
taidong.shape

In [ ]:
guanshan = pd.read_excel('../data/104年關山站_20160323.xls')
guanshan.shape

關山站的資料筆數明顯比另外兩個站少，讓我們檢查一下個測項的資料天數。

In [ ]:
pd.crosstab(guanshan['測項'],'count')

In [ ]:
6205 == (365 * 3) + (5 * 3) + 5095

關山站少了三個測項，另外有三個測項少了5天的資料，如果補上這些資料，的確就跟花蓮、台東站一樣了。

## 資料清理

接下來，我們要把遺失值換成 np.nan，然後把雨量的 NR 換成 0。

In [ ]:
def detect_epa_nan(x):
    ''' Search for missing value symbol and assign np.nan '''
    if re.findall('\#|\*|x', str(x))!=[]:
        return(np.nan)
    else:
        return(x)

def detect_epa_norain(x):
    ''' Replace 'NR' (no-rain) with 0 '''
    if str(x)=='NR':
        return(0)
    else:
        return(x)

def clean_epa_station(x):
    ''' Clean up a EPA station dataset '''
    # Rename columns
    col_names = ['date','station','item','h00','h01','h02','h03','h04','h05','h06','h07','h08','h09',
                'h10','h11','h12','h13','h14','h15','h16','h17','h18','h19','h20','h21','h22','h23']
    x.columns = col_names
    # Process NA and NR
    floatdata = x.iloc[:,3:]
    floatdata = floatdata.applymap(detect_epa_nan)
    floatdata = floatdata.applymap(detect_epa_norain)
    floatdata.astype(np.float32)
    x.iloc[:,3:] = floatdata
    # Done
    return(x)

hualian = clean_epa_station(hualian)
taidong = clean_epa_station(taidong)
guanshan = clean_epa_station(guanshan)

In [ ]:
guanshan.head()

## 資料格式轉換

如果我們想看各個測項單獨的狀態，以及彼此之間的關係，最佳的資料呈現方式，是將每個測項轉換成單獨的時間序列：

In [ ]:
# Retrieve one item from EPA data and form a time series
def retrieve_epa_item(data, var):
    tmp = data.loc[data['item']==var,:]
    ts = pd.melt(tmp, id_vars=['date'], value_vars=tmp.keys()[3:], var_name='hour', value_name=var)
    ts[var] = ts[var].astype(np.float32)
    return(ts)

# Convert EPA dataset to a collection of time-series of items
def convert_epa_itemts(data):
    # All items
    items = list(set(data['item']))
    # Create the 1st dataframe
    newdata = retrieve_epa_item(data, items[0])
    # Loop through the rest of items
    for i in items[1:]:
        tmp = retrieve_epa_item(data, i)
        newdata = newdata.merge(tmp, on=['date','hour'], how='left')
    # Sort with date-hour to make the time-series in order
    newdata = newdata.sort_values(['date', 'hour'])
    # Done
    return(newdata)

# Do the conversion
hl = convert_epa_itemts(hualian)
td = convert_epa_itemts(taidong)
gs = convert_epa_itemts(guanshan)

如此一來，我們可以很快的計算某個單獨測項有多少個遺失值。

In [ ]:
len(td) - td['O3'].count()

我們也可以用 [`DataFrame.apply`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.apply.html) 一次計算每個測項的遺失值數量。

In [ ]:
td.apply(lambda x: len(x)-x.count())

In [ ]:
# 三個測站的測項不完全相同
print(hl.keys())
print(td.keys())
print(gs.keys())

轉換成新格式後，我們可以利用 pandas.DataFrame.corr() 快速的計算同個測站不同測項的相關矩陣

In [ ]:
td.iloc[:,2:].corr()

也可以計算不同測站品項之間的相關性：

In [ ]:
pm25 = pd.DataFrame({'hualian':hl['PM2.5'], 'taidong':td['PM2.5'], 'guanshan':gs['PM2.5']})
pm25.corr()

把單一測項轉換成時間序列之後，也可以利用 `pandas.DataFrame.interpolate()` 做內插來取代遺失值。

In [ ]:
td['NOx'].describe()

In [ ]:
tdo3_int = td['NOx'].interpolate()
tdo3_int.describe()

在新的資料格式裡，我們也可以用 pandas.pivot_table() 來看測項之間的關係

In [ ]:
tmp = td.loc[:,['PM2.5','RH','WIND_SPEED']]
tmp['RH'] = pd.cut(tmp['RH'], bins=[0., 25., 50., 75., 100.])
tmp['WIND_SPEED'] = pd.qcut(tmp['WIND_SPEED'], 4)
#tmp.head()
tmp.pivot_table(columns=['RH'])

## 額外挑戰

## 在目錄下尋找特定檔案

前面的示範，都是處理少數檔案的情況，但是環保署的全部資料包含76個測站，雖然 copy-and-paste 76次也可以解決問題，但是 python 提供了更方便的工具 `os.walk()`，讓我們可以「遊走」指定資料夾底下的子目錄，然後用 `str.endswith()` 來尋找所有的 `.xls` 檔案。

In [ ]:
import os

def find_xls_files(path):
    urls = []
    for root, dirs, files in os.walk(path):
        for fname in files:
            if(fname.endswith(".xls")):
                urls.append(os.path.join(root, fname))
    return(urls)

epafiles = find_xls_files('../data/104_HOUR_00_20160323/')
epafiles[:5]

我們可以透過這個方法，把所有測站的資料都一次做處理。

In [ ]:
# Collect data from all files
data = []
nancounts = []
for f in epafiles:
    tmp = pd.read_excel(f)
    tmp = clean_epa_station(tmp)
    tmp = convert_epa_itemts(tmp)
    nancounts.append(tmp.apply(lambda x: len(x)-x.count()))
    data.append(tmp)

nancounts = pd.concat(nancounts, axis=1).T
nancounts.head()

然後我們可以很快的計算每個測站、每個測項遺失值的數量。

In [ ]:
nancounts['station'] =(epafiles)
nancounts.set_index('station', inplace=True)
nan_items = nancounts.apply(lambda x: len(x)-x.count())
nan_items

In [ ]:
nan_stations = nancounts.T.apply(lambda x: len(x)-x.count())
nan_stations.head()

In [ ]:
nan_stations.loc[nan_stations>6]

透過缺失值特徵的分析，我們可以思考如何建置比較長期的資料庫：我們希望有盡可能多的測站和測項，以及盡可能長的資料時間，透過合理的工具將資料補齊，以進一步位我們好奇的科學問題做出解答。